In [1]:
import aesara.tensor as at
import jax.numpy as jnp
import numpy as np
import jax

In [2]:
jax_key = jax.random.PRNGKey(1701)
jax_key

DeviceArray([   0, 1701], dtype=uint32)

In [3]:
[jnp.log((1/6) / (1-(1/6)))]*6

[DeviceArray(-1.609438, dtype=float32, weak_type=True),
 DeviceArray(-1.609438, dtype=float32, weak_type=True),
 DeviceArray(-1.609438, dtype=float32, weak_type=True),
 DeviceArray(-1.609438, dtype=float32, weak_type=True),
 DeviceArray(-1.609438, dtype=float32, weak_type=True),
 DeviceArray(-1.609438, dtype=float32, weak_type=True)]

In [4]:
logits = jnp.array([jnp.log((1/6) / (1-(1/6)))]*6)
jax.random.categorical(jax_key, logits, shape=(10,))

DeviceArray([1, 1, 5, 2, 2, 3, 2, 3, 5, 0], dtype=int32)

In [5]:
def multinomial_sample_fn(rng, size, dtype=jax.numpy.int32, *parameters):
    """add sampling functionality"""

    n, p = parameters
    n_max = jax.numpy.max(n)
    size = size or p.shape[:-1]
    
    logits = jax.scipy.special.logit(p)
    indices = jax.random.categorical(jax_key, logits, shape=(n_max,) + size)
    one_hot = jax.nn.one_hot(indices, p.shape[0])
    samples = jax.numpy.sum(one_hot, axis=0, dtype=dtype, keepdims=False)
    
    return indices, samples

In [17]:
indices, samples = multinomial_sample_fn(
    jax_key,
    (2,),
    jnp.int32, 
    10, 
    jnp.array([1/6]*6)
    )

In [18]:
indices, samples

(DeviceArray([[3, 2],
              [4, 2],
              [2, 3],
              [1, 3],
              [1, 4],
              [2, 2],
              [5, 0],
              [4, 4],
              [0, 5],
              [2, 2]], dtype=int32),
 DeviceArray([[1, 2, 3, 1, 2, 1],
              [1, 0, 4, 2, 2, 1]], dtype=int32))

In [59]:
shape = jax.numpy.array([10.], dtype=jax.numpy.int32)
p = jnp.array([1/6]*6)
n_max = jax.numpy.max(10)
size = None or p.shape[:-1]

s = jax.numpy.cumsum(jnp.array([1/6]*6))
r = jax.random.uniform(jax_key, shape=(n_max,) + size + (1,)) ## 10x1

# 10x6 bool matrix indicating if label r > s (cum. probs.)
bool_matrix = s < r
jax.numpy.sum(bool_matrix, axis=-1)

DeviceArray([4, 2, 2, 5, 2, 1, 5, 5, 0, 3], dtype=int32)

In [62]:
p.shape[:-1]

()